In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as pltb
from sklearn.model_selection import train_test_split


In [2]:
df=pd.read_csv('../Data/sample_labels.csv')

In [3]:
df=df[['Image Index','Finding Labels']]

In [4]:
df.head()

,Image Index,Finding Labels
0,00000013_005.png,Emphysema|Infiltration|Pleural_Thickening|Pneu...
1,00000013_026.png,Cardiomegaly|Emphysema
2,00000017_001.png,No Finding
3,00000030_001.png,Atelectasis
4,00000032_001.png,Cardiomegaly|Edema|Effusion


In [5]:
df['Diseases'] = df['Finding Labels'].apply(lambda x: x.split('|'))

In [6]:
df.head()

,Image Index,Finding Labels,Diseases
0,00000013_005.png,Emphysema|Infiltration|Pleural_Thickening|Pneu...,"[Emphysema, Infiltration, Pleural_Thickening, ..."
1,00000013_026.png,Cardiomegaly|Emphysema,"[Cardiomegaly, Emphysema]"
2,00000017_001.png,No Finding,[No Finding]
3,00000030_001.png,Atelectasis,[Atelectasis]
4,00000032_001.png,Cardiomegaly|Edema|Effusion,"[Cardiomegaly, Edema, Effusion]"


In [7]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['Labels'] = label_encoder.fit_transform(df['Finding Labels'])

In [8]:
df.head(10)

,Image Index,Finding Labels,Diseases,Labels
0,00000013_005.png,Emphysema|Infiltration|Pleural_Thickening|Pneu...,"[Emphysema, Infiltration, Pleural_Thickening, ...",190
1,00000013_026.png,Cardiomegaly|Emphysema,"[Cardiomegaly, Emphysema]",101
2,00000017_001.png,No Finding,[No Finding],234
3,00000030_001.png,Atelectasis,[Atelectasis],0
4,00000032_001.png,Cardiomegaly|Edema|Effusion,"[Cardiomegaly, Edema, Effusion]",88
5,00000040_003.png,Consolidation|Mass,"[Consolidation, Mass]",133
6,00000042_002.png,No Finding,[No Finding],234
7,00000057_001.png,No Finding,[No Finding],234
8,00000061_002.png,Effusion,[Effusion],154
9,00000061_019.png,No Finding,[No Finding],234


In [9]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [10]:
def construct_image_paths(image_folder, image_indices):
    return [os.path.join(image_folder, img) for img in image_indices]

image_folder = '../Data/images2'

train_image_paths = construct_image_paths(image_folder, train_df['Image Index'])
test_image_paths = construct_image_paths(image_folder, test_df['Image Index'])


In [11]:
import cv2
def load_and_preprocess_images(image_paths, labels):
    images = []
    for path, label in zip(image_paths, labels):
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  
        img = cv2.resize(img, (224, 224))  
        img = img / 255.0  
        images.append(img)
    return np.array(images), labels


In [12]:
X_train, y_train = load_and_preprocess_images(train_image_paths, train_df['Labels'])
X_test, y_test = load_and_preprocess_images(test_image_paths, test_df['Labels'])

In [13]:
pip install tensorflow


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator


2023-11-28 21:27:39.929215: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-28 21:27:40.243320: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-28 21:27:40.243367: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-28 21:27:40.291617: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-28 21:27:40.387801: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-28 21:27:40.388760: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [15]:
base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [16]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [17]:
for layer in base_model.layers:
    layer.trainable = False

In [18]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [19]:
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(label_encoder.classes_), activation='softmax')  
])

In [20]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [21]:
#history = model.fit(X_train, y_train, epochs=4, validation_data=(X_test, y_test))